In [1]:
import numpy as np
from scipy import stats
import pickle
import pandas as pd
import os
import matplotlib.pyplot as plt
import SimpleITK as sitk
from dipy.align.transforms import (TranslationTransform3D,
                                   RigidTransform3D,
                                   AffineTransform3D)
from dipy.align.imaffine import (transform_centers_of_mass,
                                 AffineMap,
                                 MutualInformationMetric,
                                 AffineRegistration)
from ipywidgets import interact, fixed

C:\ProgramData\Anaconda3\lib\site-packages\dicom\__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)
C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def read_kImage(path):
    # Reads the image using SimpleITK
    itkimage = sitk.ReadImage(path)
    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    ct_scan = sitk.GetArrayFromImage(itkimage)
    return ct_scan

In [3]:
def extract_voxel_data(series_path):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames( series_path )
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    image_3d = sitk.GetArrayFromImage(image)
    return image_3d

In [4]:
def extract_slice_data(img_3d, ijk):
    state = False
    k_string = ijk.split()[2]
    max_bandwith = img_3d.shape[0]
    possible_k = int (k_string)
    if (possible_k >= max_bandwith):
        possible_k = max_bandwith -1
        slice_index = possible_k
        state = True
        print("The ijk requested is {}, there was an error since the shapes of the image is {}".format(ijk,str(img_3d.shape)))
    else:
        slice_index = possible_k
        print("The ijk requested is {},OK since shapes of the image is {}".format(ijk,str(img_3d.shape)))
    return state, img_3d[slice_index,:,:]

In [5]:
def generate_tables():
    basedir = "../1/train/ProstateX-TrainingLesionInformationv2/ProstateX-TrainingLesionInformationv2"
    findingsfile = "{}/{}".format(basedir,"ProstateX-Findings-Train.csv")
    imagesfile = "{}/{}".format(basedir,"ProstateX-Images-Train.csv")
    kimagesfile = "{}/{}".format(basedir,"ProstateX-Images-KTrans-Train.csv")
    #Generamos las 3 tablas básicas a partir de los archivos.cvs
    findings_table = pd.read_csv(findingsfile)
    images_table = pd.read_csv(imagesfile)
    kimages_table = pd.read_csv(kimagesfile)
    #Operación de inner join entre tablas.
    join_kimages_table = pd.merge(left=findings_table, right= kimages_table, how ="inner", on=["ProxID","fid","pos"])
    join_images_table =  pd.merge(left=findings_table, right=  images_table, how ="inner", on=["ProxID","fid","pos"])
    #agregamos una columna para insertar la imagen
    join_images_table["MRI"] = pd.Series(np.zeros((len(join_images_table),2,2,2)).tolist(), index= join_images_table.index)
    join_images_table["MRI3D"] = pd.Series(np.zeros((len(join_images_table),2,2,2)).tolist(), index= join_images_table.index)
    join_images_table["MRI3DRegistered"] = pd.Series(np.zeros((len(join_images_table),2,2,2)).tolist(), index= join_images_table.index)
    join_images_table["ijkRegistered"] = pd.Series(False, index= join_images_table.index)
    join_images_table["MRIPatch3DRegistered"] = pd.Series(np.zeros((len(join_kimages_table),2,2,2)).tolist(), index= join_kimages_table.index)
    join_images_table["MRIPatch3D"] = pd.Series(np.zeros((len(join_kimages_table),2,2,2)).tolist(), index= join_kimages_table.index)
    join_images_table["State"] = pd.Series(False, index= join_images_table.index)
    join_kimages_table["Kimg"] = pd.Series(np.zeros((len(join_kimages_table),2,2,2)).tolist(), index= join_kimages_table.index)
    join_kimages_table["Kimg3D"] = pd.Series(np.zeros((len(join_kimages_table),2,2,2)).tolist(), index= join_kimages_table.index)
    join_kimages_table["Kimg3DRegistered"] = pd.Series(np.zeros((len(join_kimages_table),2,2,2)).tolist(), index= join_kimages_table.index)
    join_kimages_table["KimgPatch3D"] = pd.Series(np.zeros((len(join_kimages_table),2,2,2)).tolist(), index= join_kimages_table.index)
    join_kimages_table["State"] = pd.Series(False, index= join_kimages_table.index)
    return join_images_table, join_kimages_table

In [6]:
def set_MRI_value(index,image3d,state):
    window = 40
    row = join_images_table.iloc[index]
    join_images_table.at[index,"MRI3D"] = image3d
    join_images_table.at[index,"State"] = state

def set_kImg_value(index,image3d,state):
    window = 40
    row = join_kimages_table.iloc[index]
    join_kimages_table.at[index,"Kimg3D"] = image3d
    join_kimages_table.at[index,"State"] = state

In [7]:
def insert_MRI_images(join_images_table):
    for row in range (len(join_images_table)):
        patient_id = join_images_table.ProxID.iloc[row]
        series_name = join_images_table.DCMSerDescr.iloc[row]
        series_id = join_images_table.DCMSerNum.iloc[row]
        slice_ijk = join_images_table.ijk.iloc[row]
        path = "../1/Train/PROSTATEx/{}/".format(patient_id)#Selects the Patient ID
        path = "{}/{}".format(path,os.listdir(path)[0])#Selects the default folder inside the patient
        sequences = os.listdir(path) # Selects all the sequences
        sequence = [ sequence for sequence in sequences if sequence[0:len(str(series_id))] == str(series_id) ] [0]
        # Selects the only sequence that matches the condition.
        path = "{}/{}".format(path,sequence)
        image_3d = extract_voxel_data(path)
        state, image_2d = extract_slice_data(image_3d,slice_ijk)
        set_MRI_value(row,image_3d,state)
        print("Se ha cargado la imagen MRI2D y MRI3D #{} para el paciente {} serie: {} ".format(row,patient_id,series_name))
    del path, patient_id, series_name, series_id,sequence,sequences, image_3d, image_2d

In [8]:
def insert_k_images():
    for row in range(len(join_kimages_table)):
        path = "../1/Train/ProstateXKtrains-train-fixed"
        patient_id = join_kimages_table.ProxID.iloc[row]
        ijk = join_kimages_table.ijk.iloc[row]
        path = "{}/{}".format(path,patient_id)
        path = "{}/{}-Ktrans.mhd".format(path,patient_id)
        kimage_3d = read_kImage(path)
        state, kimage_2d = extract_slice_data(kimage_3d,ijk)
        set_kImg_value(row,kimage_3d,state)
        print("Se ha cargado la Kimagen2d y la kimagen3d #{} para el paciente {} ".format(row,patient_id))
        del path,patient_id,kimage_3d,state

In [9]:
join_images_table, join_kimages_table =  generate_tables()


In [10]:
insert_MRI_images(join_images_table)

The ijk requested is 36 72 9,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #0 para el paciente ProstateX-0000 serie: ep2d_diff_tra_DYNDIST_ADC 
The ijk requested is 36 72 9,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #1 para el paciente ProstateX-0000 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 36 72 9,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #2 para el paciente ProstateX-0000 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 36 72 9,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #3 para el paciente ProstateX-0000 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 36 72 9,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #4 para el paciente ProstateX-0000 serie: ep2d_diff_tra_DYNDISTCALC_BVAL 
The ijk requested is 139 166 12,OK since shapes of the image is (15, 320, 320)
Se ha cargado la i

The ijk requested is 54 68 11,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #48 para el paciente ProstateX-0003 serie: ep2d_diff_tra_DYNDISTCALC_BVAL 
The ijk requested is 149 130 0,OK since shapes of the image is (13, 256, 256)
Se ha cargado la imagen MRI2D y MRI3D #49 para el paciente ProstateX-0003 serie: t2_localizer 
The ijk requested is 199 154 14,OK since shapes of the image is (17, 320, 320)
Se ha cargado la imagen MRI2D y MRI3D #50 para el paciente ProstateX-0003 serie: t2_tse_cor 
The ijk requested is 168 165 3,OK since shapes of the image is (19, 320, 320)
Se ha cargado la imagen MRI2D y MRI3D #51 para el paciente ProstateX-0003 serie: t2_tse_sag 
The ijk requested is 240 205 12,OK since shapes of the image is (21, 384, 384)
Se ha cargado la imagen MRI2D y MRI3D #52 para el paciente ProstateX-0003 serie: t2_tse_tra 
The ijk requested is 80 69 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #53 para

The ijk requested is 50 69 11,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #100 para el paciente ProstateX-0006 serie: ep2d_diff_tra_DYNDIST_ADC 
The ijk requested is 50 69 11,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #101 para el paciente ProstateX-0006 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 50 69 11,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #102 para el paciente ProstateX-0006 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 50 69 11,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #103 para el paciente ProstateX-0006 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 50 69 11,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #104 para el paciente ProstateX-0006 serie: ep2d_diff_tra_DYNDISTCALC_BVAL 
The ijk requested is 144 146 0,OK since shapes of the image is (3, 256, 256)
Se ha

The ijk requested is 44 64 13,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #146 para el paciente ProstateX-0009 serie: ep2d_diff_tra_DYNDIST_ADC 
The ijk requested is 44 64 13,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #147 para el paciente ProstateX-0009 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 44 64 13,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #148 para el paciente ProstateX-0009 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 44 64 13,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #149 para el paciente ProstateX-0009 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 44 64 13,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #150 para el paciente ProstateX-0009 serie: ep2d_diff_tra_DYNDISTCALC_BVAL 
The ijk requested is 147 162 1,OK since shapes of the image is (3, 256, 256)
Se ha

The ijk requested is 39 57 11,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #192 para el paciente ProstateX-0012 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 39 57 11,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #193 para el paciente ProstateX-0012 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 39 57 11,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #194 para el paciente ProstateX-0012 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 39 57 11,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen MRI2D y MRI3D #195 para el paciente ProstateX-0012 serie: ep2d_diff_tra_DYNDISTCALC_BVAL 
The ijk requested is 139 126 2,OK since shapes of the image is (3, 256, 256)
Se ha cargado la imagen MRI2D y MRI3D #196 para el paciente ProstateX-0012 serie: t2_loc sag 
The ijk requested is 139 124 0,OK since shapes of the image is (13, 256, 256)
Se ha cargado la i

...

The ijk requested is 74 65 10,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3679 para el paciente ProstateX-0201 serie: tfl_3d dynamisch fast 
The ijk requested is 74 65 10,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3680 para el paciente ProstateX-0201 serie: tfl_3d dynamisch fast 
The ijk requested is 74 65 10,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3681 para el paciente ProstateX-0201 serie: tfl_3d dynamisch fast 
The ijk requested is 74 65 10,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3682 para el paciente ProstateX-0201 serie: tfl_3d dynamisch fast 
The ijk requested is 74 65 10,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3683 para el paciente ProstateX-0201 serie: tfl_3d dynamisch fast 
The ijk requested is 74 65 10,OK since shapes of the image is (12, 128, 128)
Se ha ca

The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3724 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3725 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3726 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3727 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3728 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado 

The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3773 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3774 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3775 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3776 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3777 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado 

The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3819 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3820 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3821 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3822 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3823 para el paciente ProstateX-0203 serie: tfl_3d PD reference 
The ijk requested is 70 54 8,OK since shapes of the image is (20, 106, 128)
Se ha cargado la

The ijk requested is 71 65 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3866 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 71 65 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3867 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 71 65 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen MRI2D y MRI3D #3868 para el paciente ProstateX-0203 serie: tfl_3d PD reference 


In [11]:
insert_k_images()

The ijk requested is 56 75 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #0 para el paciente ProstateX-0000 
The ijk requested is 51 61 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #1 para el paciente ProstateX-0001 
The ijk requested is 52 73 11,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #2 para el paciente ProstateX-0002 
The ijk requested is 68 72 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #3 para el paciente ProstateX-0002 
The ijk requested is 80 69 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #4 para el paciente ProstateX-0003 
The ijk requested is 51 54 13,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #5 para el paciente ProstateX-0003 
The ijk requested is 74 56 6,OK since shapes of the image is (16, 128, 12

The ijk requested is 54 63 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #58 para el paciente ProstateX-0037 
The ijk requested is 56 74 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #59 para el paciente ProstateX-0037 
The ijk requested is 53 63 9,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #60 para el paciente ProstateX-0038 
The ijk requested is 76 71 9,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #61 para el paciente ProstateX-0038 
The ijk requested is 54 74 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #62 para el paciente ProstateX-0038 
The ijk requested is 46 75 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #63 para el paciente ProstateX-0039 
The ijk requested is 50 63 10,OK since shapes of the image is (16, 12

The ijk requested is 69 82 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #116 para el paciente ProstateX-0081 
The ijk requested is 57 63 12,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #117 para el paciente ProstateX-0082 
The ijk requested is 67 73 12,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #118 para el paciente ProstateX-0083 
The ijk requested is 63 69 14,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #119 para el paciente ProstateX-0083 
The ijk requested is 74 65 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #120 para el paciente ProstateX-0084 
The ijk requested is 56 66 4,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #121 para el paciente ProstateX-0084 
The ijk requested is 46 73 9,OK since shapes of the image is 

The ijk requested is 56 57 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #175 para el paciente ProstateX-0116 
The ijk requested is 73 59 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #176 para el paciente ProstateX-0117 
The ijk requested is 72 70 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #177 para el paciente ProstateX-0117 
The ijk requested is 65 62 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #178 para el paciente ProstateX-0118 
The ijk requested is 70 74 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #179 para el paciente ProstateX-0118 
The ijk requested is 73 66 5,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #180 para el paciente ProstateX-0119 
The ijk requested is 67 61 3,OK since shapes of the image is (1

The ijk requested is 63 58 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #231 para el paciente ProstateX-0145 
The ijk requested is 71 74 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #232 para el paciente ProstateX-0146 
The ijk requested is 52 78 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #233 para el paciente ProstateX-0147 
The ijk requested is 66 59 9,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #234 para el paciente ProstateX-0147 
The ijk requested is 67 55 3,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #235 para el paciente ProstateX-0148 
The ijk requested is 49 70 4,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #236 para el paciente ProstateX-0148 
The ijk requested is 68 79 8,OK since shapes of the image is (

The ijk requested is 51 59 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #288 para el paciente ProstateX-0177 
The ijk requested is 57 69 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #289 para el paciente ProstateX-0177 
The ijk requested is 63 56 5,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #290 para el paciente ProstateX-0177 
The ijk requested is 69 46 9,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #291 para el paciente ProstateX-0178 
The ijk requested is 50 59 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #292 para el paciente ProstateX-0179 
The ijk requested is 64 72 4,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #293 para el paciente ProstateX-0179 
The ijk requested is 47 80 9,OK since shapes of the image is (16

In [12]:
MRI_dict = {"ep2d": "echo-planar imaging two-dimensional", "t1":"t1 weighted", "tfl":"T1-weighted Fast Low Angle Shot (Turbo flash)",
            "t2":"t2 weighted", "diff":"Diffusion-weighted magnetic resonance imaging","perfusie":"perfusion-weighted imaging",
            "perfusie_t1" : "Dynamic contrast enhanced (DCE)", "tra": "Transverse", "cor" : "Coronal", "sag":"Sagittal",
            "adc":"Apparent Diffusion Coeffitient (ADC)", "tse":"Turbo spin echo", "localizer":"Scout scan"
           }

In [13]:

def get_image_description(DCM_description):
    DCM_description = str.lower(DCM_description)
    DCM_description = DCM_description.replace(" ","_")
    print("This is the description for the sequence: {}".format(DCM_description))
    DCM_description.replace(" ","_")
    values = DCM_description.split("_")
    for i in range(len(values)):
        word = values[i]
        if (word == "perfusie" and values[i + 1] =="t1"):
            word = "perfusie_t1"
        try:
            print(MRI_dict[word])
        except:
            print(str.upper(word))
value = "ep2d_diff_tra_DYNDIST_ADC"
get_image_description(value)

This is the description for the sequence: ep2d_diff_tra_dyndist_adc
echo-planar imaging two-dimensional
Diffusion-weighted magnetic resonance imaging
Transverse
DYNDIST
Apparent Diffusion Coeffitient (ADC)


In [14]:
def plot_sequence(description,image,z_value):
    plt.figure(figsize=(8,8))
    plt.title(description)
    plt.imshow(image[z_value])
    plt.colorbar()

In [15]:
def get_patient_finding_descriptions(patient_finding_table):
    all_descriptions = patient_finding_table.DCMSerDescr.unique()
    for description in all_descriptions:
        get_image_description(description)
        print("==============================================================================================")

In [16]:
def get_mri_images_of_patient_and_finding(patient, finding):
    result = join_images_table[(join_images_table["ProxID"] == patient)]
    result = result[result["fid"] == finding]
    return result

In [17]:
def get_kimg_of_patient_and_finding(patient, finding):
    result = join_kimages_table[join_kimages_table.ProxID == patient]
    result = result[result.fid == finding]
    return result

In [18]:
print("MRI 3D Images")
all_patients = join_images_table.ProxID.unique()
patient = all_patients[np.random.randint(0,len(all_patients))]
finding = int(input("Please enter a number of finding (finding id) #"))
print("The patient of study is {}".format(patient))
patient_finding_table = get_mri_images_of_patient_and_finding(patient, finding)
print("There is a total of {} sequences".format(len(patient_finding_table)))
get_patient_finding_descriptions(patient_finding_table)
display(patient_finding_table[["ProxID", "fid", "zone", "ClinSig", "ijk", "DCMSerDescr", "Name", "MRI3D"]])

MRI 3D Images
Please enter a number of finding (finding id) #1
The patient of study is ProstateX-0193
There is a total of 11 sequences
This is the description for the sequence: ep2d_diff_tra_dyndist_adc
echo-planar imaging two-dimensional
Diffusion-weighted magnetic resonance imaging
Transverse
DYNDIST
Apparent Diffusion Coeffitient (ADC)
This is the description for the sequence: ep2d_diff_tra_dyndist
echo-planar imaging two-dimensional
Diffusion-weighted magnetic resonance imaging
Transverse
DYNDIST
This is the description for the sequence: ep2d_diff_tra_dyndistcalc_bval
echo-planar imaging two-dimensional
Diffusion-weighted magnetic resonance imaging
Transverse
DYNDISTCALC
BVAL
This is the description for the sequence: t2_loc_sag
t2 weighted
LOC
Sagittal
This is the description for the sequence: t2_localizer
t2 weighted
Scout scan
This is the description for the sequence: t2_tse_cor
t2 weighted
Turbo spin echo
Coronal
This is the description for the sequence: t2_tse_sag
t2 weighted
T

ProxID  fid zone  ClinSig         ijk  \
3348  ProstateX-0193    1   AS    False     58 53 8   
3349  ProstateX-0193    1   AS    False     58 53 8   
3350  ProstateX-0193    1   AS    False     58 53 8   
3351  ProstateX-0193    1   AS    False     58 53 8   
3352  ProstateX-0193    1   AS    False     58 53 8   
3353  ProstateX-0193    1   AS    False   116 130 2   
3354  ProstateX-0193    1   AS    False   136 122 0   
3355  ProstateX-0193    1   AS    False   165 153 1   
3356  ProstateX-0193    1   AS    False  127 147 10   
3357  ProstateX-0193    1   AS    False   184 145 8   
3358  ProstateX-0193    1   AS    False     62 49 7   

                         DCMSerDescr                             Name  \
3348       ep2d_diff_tra_DYNDIST_ADC       ep2d_diff_tra_DYNDIST_ADC0   
3349           ep2d_diff_tra_DYNDIST           ep2d_diff_tra_DYNDIST0   
3350           ep2d_diff_tra_DYNDIST           ep2d_diff_tra_DYNDIST1   
3351           ep2d_diff_tra_DYNDIST           ep2d_diff_tra_DYNDIST2   
3352  ep2d_diff_tra_DYNDISTCALC_BVAL  ep2d_diff_tra_DYNDISTCALC_BVAL0   
3353                      t2_loc sag                      t2_loc_sag0   
3354                    t2_localizer                    t2_localizer0   
3355                      t2_tse_cor                      t2_tse_cor0   
3356                      t2_tse_sag                      t2_tse_sag0   
3357                      t2_tse_tra                      t2_tse_tra0   
3358    tfl_3d PD ref_tra_1.5x1.5_t3    tfl_3d_PD_ref_tra_1_5x1_5_t30   

                                                  MRI3D  
3348  [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...  
3349  [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...  
3350  [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...  
3351  [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...  
3352  [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...  
3353  [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...  
3354  [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...  
3355  [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...  
3356  [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...  
3357  [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...  
3358  [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...

In [19]:
indexes = patient_finding_table.index
for index in indexes:
    row = join_images_table.iloc[index]
    image = row.MRI3D
    position = row.ijk
    description = row.DCMSerDescr
    description = "{} pos: {}".format(description,position)
    interact(plot_sequence,description=fixed(description), z_value =(0,len(image)-1), image = fixed(image) )

interactive(children=(IntSlider(value=9, description='z_value', max=18), Output()), _dom_classes=('widget-inte…

interactive(children=(IntSlider(value=28, description='z_value', max=56), Output()), _dom_classes=('widget-int…

interactive(children=(IntSlider(value=28, description='z_value', max=56), Output()), _dom_classes=('widget-int…

interactive(children=(IntSlider(value=28, description='z_value', max=56), Output()), _dom_classes=('widget-int…

interactive(children=(IntSlider(value=9, description='z_value', max=18), Output()), _dom_classes=('widget-inte…

...

interactive(children=(IntSlider(value=6, description='z_value', max=12), Output()), _dom_classes=('widget-inte…

interactive(children=(IntSlider(value=7, description='z_value', max=14), Output()), _dom_classes=('widget-inte…

interactive(children=(IntSlider(value=9, description='z_value', max=18), Output()), _dom_classes=('widget-inte…

interactive(children=(IntSlider(value=9, description='z_value', max=18), Output()), _dom_classes=('widget-inte…

interactive(children=(IntSlider(value=7, description='z_value', max=15), Output()), _dom_classes=('widget-inte…

In [20]:
patient_finding_ktable = get_kimg_of_patient_and_finding(patient=patient, finding=finding)
display(patient_finding_ktable)

ProxID  fid                      pos zone  ClinSig  \
315  ProstateX-0193    1  7.42249 10.9888 10.3919   AS    False   

                                           WorldMatrix      ijk  \
315  1.5,3.07655e-010,5.8488e-017,-85.0065,-3.00363...  62 49 7   

                                                  Kimg  \
315  [[[0.0, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.0, ...   

                                                Kimg3D  \
315  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...   

                                      Kimg3DRegistered  \
315  [[[0.0, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.0, ...   

                                           KimgPatch3D  State  
315  [[[0.0, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.0, ...  False

In [21]:
print("Kimages")
indexes2 = patient_finding_ktable.index
for index in indexes2:
    row = join_kimages_table.iloc[index]
    image = row.Kimg3D
    position = row.ijk
    description = "Kimage"
    description = "{} pos: {}".format(description,position)
    interact(plot_sequence,description=fixed(description), z_value =(0,len(image)-1), image = fixed(image) )

Kimages


interactive(children=(IntSlider(value=7, description='z_value', max=15), Output()), _dom_classes=('widget-inte…

In [22]:
max_finding = max(join_images_table.fid)
min_finding = min(join_images_table.fid)
print("The maximing id for finding is: {}".format(max_finding))
print("The minimun id for finding is: {}".format(min_finding))
print("These were the patients available for the experiment:")
patients = join_images_table.ProxID.unique()
for patient in patients:
    print(patient)
    
get_patient_finding_descriptions(join_images_table)

The maximing id for finding is: 4
The minimun id for finding is: 0
These were the patients available for the experiment:
ProstateX-0000
ProstateX-0001
ProstateX-0002
ProstateX-0003
ProstateX-0004
ProstateX-0005
ProstateX-0006
ProstateX-0007
ProstateX-0008
ProstateX-0009
ProstateX-0010
ProstateX-0011
ProstateX-0012
ProstateX-0013
ProstateX-0014
ProstateX-0015
ProstateX-0016
ProstateX-0017
ProstateX-0018
ProstateX-0019
ProstateX-0020
ProstateX-0021
ProstateX-0022
ProstateX-0023
ProstateX-0024
ProstateX-0025
ProstateX-0026
ProstateX-0027
ProstateX-0028
ProstateX-0029
ProstateX-0030
ProstateX-0031
ProstateX-0032
ProstateX-0033
ProstateX-0034
ProstateX-0035
ProstateX-0036
ProstateX-0037
ProstateX-0038
ProstateX-0039
ProstateX-0040
ProstateX-0041
ProstateX-0042
ProstateX-0043
ProstateX-0044
ProstateX-0045
ProstateX-0046
ProstateX-0047
ProstateX-0048
ProstateX-0049
ProstateX-0050
ProstateX-0051
ProstateX-0052
ProstateX-0053
ProstateX-0054
ProstateX-0055
ProstateX-0056
ProstateX-0057
ProstateX